<a href="https://colab.research.google.com/github/Prkhar05/Tiny_Text2SQL/blob/main/queries_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!huggingface-cli login

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!pip install torch torchvision torchaudio
!pip install bitsandbytes
!pip install accelerate
!pip install tqdm

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from tqdm import tqdm
import pandas as pd


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")
model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-128k-instruct", load_in_4bit=True, device_map="auto",trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


configuration_phi3.py:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

In [ ]:
# schema = """
# Authors Table
# This table holds information about authors.

# author_id: Primary key, typically an integer.
# author_name: Text field for the author's name.
# Books Table
# This table contains details about books each author has written.

# book_id: Primary key, typically an integer.
# author_id: Foreign key that references author_id in the authors table. This establishes a relationship between each book and its author.
# title: Text field for the book's title.
# publication_date: Date field to store the date when the book was published.
# language: Text field to indicate the language of the book.
# is_bestseller: Boolean field indicating whether the book is a bestseller. This field is hypothetical based on one of the queries discussed.
# Sales Table
# This table records sales data for each book.

# sale_id: Primary key, typically an integer.
# book_id: Foreign key that references book_id in the books table. This links each sale to a specific book.
# units_sold: Integer field to record the number of units sold.
# sale_date: Date field that records when the sale occurred.
# Book Sales Table
# Assuming a more complex scenario where books and sales data might be separated, here's an optional additional table, which might not be necessary if sales are recorded directly in a sales table:

# book_sales_id: Primary key, integer.
# book_id: Foreign key, links to books table.
# units_sold: Integer, records the quantity of the book sold."""

In [ ]:
schema = """ CREATE TABLE Authors (
    author_id INT PRIMARY KEY,
    author_name TEXT
);

CREATE TABLE Books (
    book_id INT PRIMARY KEY,
    author_id INT,
    title TEXT,
    publication_date DATE,
    language TEXT,
    is_bestseller BOOLEAN,
    FOREIGN KEY (author_id) REFERENCES Authors(author_id)
);

CREATE TABLE Sales (
    sale_id INT PRIMARY KEY,
    book_id INT,
    units_sold INT,
    sale_date DATE,
    FOREIGN KEY (book_id) REFERENCES Books(book_id)
);

CREATE TABLE BookSales (
    book_sales_id INT PRIMARY KEY,
    book_id INT,
    units_sold INT,
    FOREIGN KEY (book_id) REFERENCES Books(book_id)
);
"""

In [ ]:
def get_prompt(user_question, table_data):
    return f"""### Task
Generate a SQL query to answer [QUESTION]{user_question}[/QUESTION]

### Database Schema
The query will run on a database with the following schema details:
{table_data}

### Answer
Given the database schema, here is the SQL query that [QUESTION]{user_question}[/QUESTION]
[SQL]"""

In [ ]:
user_question = [
    "Give me the names of author who has written atleast one book.",
    "Give me the names of author and the books they have written?",
    "Give me author who have written more than 2 books or atleast 2 books?",
    "Give me author who have written just 2 books or only 2 books or 2 books?",
    "Give me author who have written less than 2 books or max 2 books?",
    "Give me author who have written maximum number of books?",
    "Give me author with all the books names who has written maximum number of books?",
    "Give me top 3 author with all the books names who has written maximum number of books?",
    "Give me names of author who have written more books than last year?",
    "Give me names of author who have written more books in year 2012 than 2022?",
    "Give me names of author who has written atleast 2 books quarter on quater in the year 2022?",
    "Give me names of author who has written atleast 1 books month on month in the year 2022?",
    "Give me authors running sales over time, books, units sold for last 5 years for each language",
    "Give me top-ranked books or any author with more than 1000 total sales",
    "Give me list of books written by author for each language for previous and current year",
    "Give me authors running sales over time, books, units sold for last 5 years",
    "Give me authors running sales over time, books, units sold",
    "Give me list of authors, their book titles, the number of units sold for each book, and the percentile (use NTILE function) rank of each book within the context of its author",
    "Give me list of authors, their book titles, the number of units sold for each book, and the percentile rank of each book within the context of its author using NITILE",
    "Give me list of authors, their book titles, the number of units sold for each book, and the percentile rank of each book within the context of its author.",
    "Give me best-selling books for each author, and how many best-selling books each author has",
    "Give me list of authors who have more than one best-selling book, along with the count of their best-sellers",
    "Give me authors who have more than one best-selling book, along with the count of their best-sellers from author and books table",
    "Give me authors who have more than one best-selling book, along with the count of their best-sellers",
    "Give me list of authors, along with their sales for current year and their sales for the previous year, and the percentage growth or decline in sales from the previous year to the current year",
    "Show the name of each author along with the title of their top-selling book and the total units sold for that book from author table",
    "Show the name of each author along with the title of their top-selling book and the total units sold for that book from author, sales and books table",
    "Show the name of each author along with the title of their top-selling book and the total units sold for that book",
    "Show me author's name along with the details of their top-selling book",
    "Give me author's name along with their top-selling books across different years in last decade",
    "Give me top 5 author names in each language who have maximum sales",
    "Give me list of books by top 5 authors across different languages",
    "Give me list of books by each author divide into 5 buckets based on sales",
    "Show me the book, name of authors and year which is written by exactly 3 authors for last 5 years",
    "Give me names of author, monthwise number of books published for authors who have written atleast 2 books last year where publication date format is YYMMDD",
    "Give me names of author, quarter wise number of books published for authors who have written more books than 3 books in last 3 years where publication date format is YYMMDD",
    "Give me names of author, quarter wise number of books published for authors who have written more books than 3 books in last 3 years",
    "Give me names of author, q1 2012, q2 2012, q3 2012, q4 2012, q1 2013, q2 2013, q3 2013, q4 2013, number of books published for authors who have written more books than 3 books for 2012, 2013",
    "Give me names of author, year of issue 2012, year of issue 203, number of books published for authors who have written more books than 3 books for 2012, 2013",
    "Give me names of author, year of issue, number of books published for authors who have written more books than 3 books for 2012, 2013, 2014 and 2017",
    "Give me names of author who have written more books in year 2012 than 2022? Date column is in string format",
    "Show me top 3 author names in each country by language who have written maximum books in the year 2023 where publication date is a string datatype column with format MMDDYYYY",
    "Show me top 3 author names in each country by language who have written maximum books in the year 2023 where publication date is a string datatype column",
    "Show me bottom 10 author names and book title in each country by language and books pages who have written maximum books in the year 2023",
    "Show me top 2 author names and book title in each country by language who have written maximum books in the year 2023",
    "Show me top 3 author names in each country by language who have written maximum books in the year 2023",
    "Show me top 4 authors, grouped by country for every city, month who have written books in german and published book in all given language in all quarter of the year 2015. sort by quarter with maximum books. compare it with last 3 years.",
    "Show me top 4 authors and month who have written books in german and published book in all given language in all quarter of the year 2015. sort by quarter with maximum books. compare it with last 3 years. add author country as well",
    "Show me top 4 authors and month who have written books in german and published book in all given language in all quarter of the year 2015. sort by quarter with maximum books. compare it with last 3 years",
    "Show me top 4 authors and month who have written books in german and published book in all given language in all quarter of the year 2015. sort by quarter with maximum books. add their 2013 numbers to compare",
    "Show me top 4 authors and month who have written books in german and published book in all given language in all quarter of the year 2015. sort by quarter with maximum books",
    "Show me top 5 authors and month who have written books in english and published book in all given language in all quarter of the year 2012. sort by month with maximum books",
    "Show me top 5 authors and month who have written books in english and published book in all given language in every month of the year 2012. sort by month with maximum books",
    "Show me top 5 authors who have written books in english, hindi and marathi and published book in all given language in every month of the year 2012",
    "Show me top 5 books, name of authors which is written by maximum number of authors for last 5 years with same initials",
    "Show me top 5 books, name of authors, quarter of year which is written by maximum number of authors for last 5 years",
    "Show me the book, name of authors and year which is written by maximum number of authors for last 5 years",
    "Show me the book and name of authors which is written by maximum number of authors for last 5 years",
    "Show me the book and name of authors which is written by maximum number of authors",
    "Show me the names of the author and book where the book was written by more than 2 authors",
    "Show me the names of the author and book where the book was written by more than 2 authors"
]


In [ ]:
# List to store the results
prompts = []

# Applying the function to each item in the user_question list
for question in user_question:
    prompt = get_prompt(question, schema)
    prompts.append(prompt)

In [ ]:
prompts[0]

'### Task\nGenerate a SQL query to answer [QUESTION]Give me the names of author who has written atleast one book.[/QUESTION]\n\n### Database Schema\nThe query will run on a database with the following schema details:\n CREATE TABLE Authors (\n    author_id INT PRIMARY KEY,\n    author_name TEXT\n);\n\nCREATE TABLE Books (\n    book_id INT PRIMARY KEY,\n    author_id INT,\n    title TEXT,\n    publication_date DATE,\n    language TEXT,\n    is_bestseller BOOLEAN,\n    FOREIGN KEY (author_id) REFERENCES Authors(author_id)\n);\n\nCREATE TABLE Sales (\n    sale_id INT PRIMARY KEY,\n    book_id INT,\n    units_sold INT,\n    sale_date DATE,\n    FOREIGN KEY (book_id) REFERENCES Books(book_id)\n);\n\nCREATE TABLE BookSales (\n    book_sales_id INT PRIMARY KEY,\n    book_id INT,\n    units_sold INT,\n    FOREIGN KEY (book_id) REFERENCES Books(book_id)\n);\n\n\n### Answer\nGiven the database schema, here is the SQL query that [QUESTION]Give me the names of author who has written atleast one bo

In [ ]:
def generate_predictions(prompt):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(input_ids=input_ids, max_length=len(input_ids[0]) + 1000, num_return_sequences=1)
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    predictions = answer[len(prompt):]

    return predictions


In [ ]:
predictions_list = []
for prompt in tqdm(prompts):
    predictions_list.append(generate_predictions(prompt))

  0%|          | 0/61 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1510: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
  2%|▏         | 1/61 [00:34<34:57, 34.95s/it]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1510: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than

In [ ]:
# predictions_list = [prediction.rstrip("[/SQL]") for prediction in predictions_list]

In [ ]:
predictions_list[60]

'\n\n```sql\nSELECT DISTINCT a.author_name, b.title\nFROM Authors a\nJOIN Books b ON a.author_id = b.author_id\nWHERE (\n    SELECT COUNT(*)\n    FROM Authors\n    WHERE author_id IN (\n        SELECT author_id\n        FROM Books\n        WHERE Books.author_id = a.author_id\n    )\n) > 2;\n```\n\nThis query works as follows:\n1. It selects distinct authors and book titles from the Books and Authors tables.\n2. The `WHERE` clause uses a subquery to count the number of authors (`author_id`) associated with each book in the Books table.\n3. The subquery in the `WHERE` clause counts the number of times an `author_id` appears in the Books table, which corresponds to the number of books written by the same author.\n4. The condition `(SELECT COUNT(*) > 2)` filters out the books that have been written by more than two authors.\n5. The `DISTINCT` keyword ensures that each author-book pair is listed only once, even if the author has written multiple books that meet the criteria.\n\nPlease note 

In [ ]:
df = pd.DataFrame({'Questions': user_question, 'Predictions': predictions_list})

In [ ]:
df.head()

,Questions,Predictions
0,Give me the names of author who has written at...,SELECT author_name FROM Authors WHERE EXISTS (...
1,Give me the names of author and the books they...,"\nSELECT a.author_name, b.title\nFROM Authors ..."
2,Give me author who have written more than 2 bo...,\n\n```sql\nSELECT A.author_name\nFROM Authors...
3,Give me author who have written just 2 books o...,\n\n```sql\nSELECT a.author_name\nFROM Authors...
4,Give me author who have written less than 2 bo...,\n\n```sql\nSELECT a.author_name\nFROM Authors...


In [ ]:
df.to_csv('phi3-query_predictions.csv', index=False)